# Transcriptomic Signature of Mycovirus Infection in Yeast

## Objective
To test whether infection by specific classes of mycoviruses leads to consistent transcriptional changes across *Saccharomyces cerevisiae* strains.

## Data Sources
- RNA-seq dataset from Caudal et al. (2024) covering hundreds of yeast strains
- Mycovirus infection metadata per strain (e.g. presence/absence of Narnaviridae, Totiviridae, etc.)
- Additional phenotype metadata (growth rate, ecology, clade, etc.)

## Approach
1. Load RNA-seq expression matrix (genes × strains)
2. Group strains by virus infection status
3. Perform differential expression analysis (infected vs uninfected)
4. Run gene set enrichment (GO terms, pathways)
5. Visualize results

## Notes
- This is an exploratory analysis.
- Viral infection may correlate with strain background or ecology.
- We will interpret findings cautiously and use controls when possible.

In [7]:
import pandas as pd
from pathlib import Path
import io, csv, textwrap

# -----------------------
# Config (edit this only)
# -----------------------
FILE = "data/final_data_annotated_merged_04052022.tab"  # .tab or .tab.gz/.zip ok
# Column names (the file can have slight variations; we’ll auto-fallback too)
CAND_GENE_COLS   = ["systematic_name", "ORF"]
CAND_STRAIN_COLS = ["Strain", "Standardized_name"]
COUNT_COL        = "count"
TPM_COL          = "tpm"
CHUNK            = 250_000
OUT_DIR          = Path("./data/")  # where to write matrices

# -----------------------
# Helpers
# -----------------------
def sniff_sep(path, default="\t", sample_bytes=1_000_000):
    """
    Try to sniff delimiter from the first ~1MB. Fall back to default.
    Works with compressed; pandas handles compression by extension later.
    """
    p = Path(path)
    # For zip/gz/bz2, pandas will handle decompression;
    # We only need a guess — try reading a small block via pandas
    try:
        # First try tab quickly; if columns==1, try comma.
        df_head = pd.read_csv(p, sep="\t", nrows=5, engine="python", on_bad_lines="skip")
        if df_head.shape[1] > 1:
            return "\t"
        df_head = pd.read_csv(p, sep=",", nrows=5, engine="python", on_bad_lines="skip")
        if df_head.shape[1] > 1:
            return ","
    except Exception:
        pass
    return default

def read_header(path, sep):
    """Read just the header to see available columns."""
    df = pd.read_csv(path, sep=sep, nrows=5, engine="python", on_bad_lines="skip")
    return list(df.columns)

def pick_first(existing, candidates):
    """Pick the first candidate that exists (case sensitive)."""
    for c in candidates:
        if c in existing:
            return c
    return None

def aggregate_chunk(df, gene_col, strain_col, count_col=None, tpm_col=None):
    """Return (agg_counts, agg_tpms) grouped by (gene,strain)."""
    keys = [gene_col, strain_col]
    out_c = None
    out_t = None
    if count_col and count_col in df.columns:
        out_c = (
            df[keys + [count_col]]
            .dropna(subset=keys)
            .groupby(keys, as_index=False, sort=False)[count_col]
            .sum()
        )
    if tpm_col and tpm_col in df.columns:
        out_t = (
            df[keys + [tpm_col]]
            .dropna(subset=keys)
            .groupby(keys, as_index=False, sort=False)[tpm_col]
            .sum()
        )
    return out_c, out_t

def merge_agg(existing, new, key_cols, value_col):
    """Merge two aggregated (gene,strain,value) tables by sum."""
    if existing is None:
        return new
    if new is None or new.empty:
        return existing
    merged = pd.concat([existing, new], ignore_index=True)
    merged = merged.groupby(key_cols, as_index=False, sort=False)[value_col].sum()
    return merged

def to_wide(agg_df, gene_col, strain_col, value_col, fill=0.0, dtype="float32"):
    """Pivot long -> wide (genes × strains)."""
    if agg_df is None or agg_df.empty:
        return None
    wide = agg_df.pivot_table(index=gene_col, columns=strain_col, values=value_col, fill_value=fill, aggfunc="sum")
    # keep columns sorted for reproducibility
    wide = wide.sort_index(axis=0).sort_index(axis=1)
    return wide.astype(dtype)

# -----------------------
# Main loader
# -----------------------
def build_matrices(
    path,
    gene_cols=CAND_GENE_COLS,
    strain_cols=CAND_STRAIN_COLS,
    count_col=COUNT_COL,
    tpm_col=TPM_COL,
    chunksize=CHUNK,
    out_dir=OUT_DIR,
):
    path = Path(path)
    sep = sniff_sep(path)
    print(f"[info] Using delimiter: {repr(sep)}")

    cols = read_header(path, sep)
    print(f"[info] Header has {len(cols)} columns.")
    # Try to pick gene/strain columns
    gene_col = pick_first(cols, gene_cols)
    strain_col = pick_first(cols, strain_cols)

    # Diagnostics if not found
    if gene_col is None or strain_col is None:
        msg = textwrap.dedent(f"""
        [error] Could not find required columns.
        - gene candidates tried: {gene_cols}
        - strain candidates tried: {strain_cols}
        - header columns: {cols[:20]}{' ...' if len(cols)>20 else ''}
        """).strip()
        raise ValueError(msg)

    if count_col not in cols and tpm_col not in cols:
        raise ValueError(f"[error] Neither '{count_col}' nor '{tpm_col}' found. Header preview: {cols[:20]}")

    print(f"[info] Using columns: gene={gene_col}, strain={strain_col}, count={count_col in cols}, tpm={tpm_col in cols}")

    usecols = [gene_col, strain_col]
    if count_col in cols: usecols.append(count_col)
    if tpm_col   in cols: usecols.append(tpm_col)

    # Aggregators (long format)
    agg_counts = None
    agg_tpms   = None

    # Stream read
    try:
        # Newer pandas supports encoding_errors
        reader = pd.read_csv(
            path,
            sep=sep,
            usecols=usecols,
            chunksize=chunksize,
            dtype={gene_col:"string", strain_col:"string"},
            engine="python",
            on_bad_lines="skip",
            encoding="utf-8",
            encoding_errors="replace",   # ← keep row, replace bad bytes with �
        )
    except TypeError:
        # Older pandas: no encoding_errors param. Fallback to latin1 which maps all bytes.
        reader = pd.read_csv(
            path,
            sep=sep,
            usecols=usecols,
            chunksize=chunksize,
            dtype={gene_col:"string", strain_col:"string"},
            engine="python",
            on_bad_lines="skip",
            encoding="latin1",           # ← accepts any byte, preserves row integrity
        )

    nrows = 0
    for i, chunk in enumerate(reader, 1):
        nrows += len(chunk)
        ac, at = aggregate_chunk(chunk, gene_col, strain_col, count_col if count_col in chunk.columns else None,
                                 tpm_col if tpm_col in chunk.columns else None)
        agg_counts = merge_agg(agg_counts, ac, [gene_col, strain_col], count_col) if ac is not None else agg_counts
        agg_tpms   = merge_agg(agg_tpms,   at, [gene_col, strain_col], tpm_col)   if at is not None else agg_tpms
        if i % 10 == 0:
            print(f"[info] processed ~{nrows:,} rows...")

    print(f"[info] finished streaming ~{nrows:,} rows.")
    # Pivot to wide
    count_wide = to_wide(agg_counts, gene_col, strain_col, count_col) if agg_counts is not None else None
    tpm_wide   = to_wide(agg_tpms,   gene_col, strain_col, tpm_col)   if agg_tpms   is not None else None

    # Write to disk
    out_paths = {}
    out_dir.mkdir(parents=True, exist_ok=True)
    if count_wide is not None:
        p = out_dir / "count_matrix.csv.gz"
        count_wide.to_csv(p, compression="gzip")
        out_paths["counts"] = str(p)
        print(f"[ok] wrote counts → {p}  shape={count_wide.shape}")

    if tpm_wide is not None:
        p = out_dir / "tpm_matrix.csv.gz"
        tpm_wide.to_csv(p, compression="gzip")
        out_paths["tpm"] = str(p)
        print(f"[ok] wrote TPM    → {p}  shape={tpm_wide.shape}")

    return count_wide, tpm_wide, out_paths

# -----------------------
# Run
# -----------------------
counts_df, tpm_df, outputs = build_matrices(FILE)
counts_df.head() if counts_df is not None else tpm_df.head()

[info] Using delimiter: ','
[info] Header has 30 columns.
[info] Using columns: gene=systematic_name, strain=Strain, count=True, tpm=True
[info] processed ~2,500,000 rows...
[info] processed ~5,000,000 rows...
[info] finished streaming ~6,314,973 rows.
[ok] wrote counts → data/count_matrix.csv.gz  shape=(6454, 969)
[ok] wrote TPM    → data/tpm_matrix.csv.gz  shape=(6454, 969)


Strain,AAA,AAB,AAD,AAE,AAG,AAH,AAI,AAK,AAL,AAM,...,XTRA_DGX,XTRA_DGY,XTRA_DHB,XTRA_DHD,XTRA_DHE,XTRA_DHJ,XTRA_DHK,XTRA_DHO,XTRA_DHQ,XTRA_DXL
systematic_name,,,,,,,,,,,,,,,,,,,,,
X1-EC1118_1F14_0012g,0.0,25.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X10-EC1118_1F14_0133g,0.0,1368.0,0.0,0.0,0.0,47.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X1003-augustus_masked.YCM.7680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X1004-augustus_masked.YCM.7680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X1005-augustus_masked.YCM.7680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
